# Process Approved Variants

This notebook:
1. Loads approved variants from the CSV file
2. Copies variants to S3 with new variant IDs (original ID * 100000)
3. Updates the Redshift database
4. Updates the API image list

In [16]:
# Import the module with all the functions
from process_approved_variants import *

In [21]:
import pandas as pd

# Attempt to read the CSV file with 'ISO-8859-1' encoding to handle potential encoding issues
df = pd.read_csv('/Users/elliottoates/Desktop/streamlit-image-review/updated_100_motoring.csv')
# Drop any columns with 'Unnamed' in their name
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
# Replace string '1' with 'approved'
df['review_result'] = df['review_result'].replace('1', 'approved')
# Filter out rejected images
df_rejected = df[df['review_result'] == 'rejected']
df['id'] = df['id'].astype(int)
df.head(5)

,id,email_subject,category_name,vertical,sub_category_name,revenue_last_7_days,revenue_rank,image_id_pos_0,image_url_pos_0,extension,status,s3_url,token_info,processed_timestamp,prompt,prompt_source,review_result,review_notes
0,16268711,Supercar & Sports Car Driving Experience with ...,Activities,LOCAL,Motoring,297.63,1,660429,https://static.wowcher.co.uk/images/deal/16268...,jpg,success,https://static.wowcher.co.uk/images/deal/16268...,"{'Total tokens': '8824', 'Input tokens': '2616...",2025-07-15 16:03:20.408076,<imageRequest>\n <metadata>\n <title>Super...,motoring,rejected,NaN
1,22854381,Formula Renault Racing Car Driving Experience ...,Activities,LOCAL,Motoring,260.99,2,852549,https://static.wowcher.co.uk/images/deal/22854...,jpg,success,https://static.wowcher.co.uk/images/deal/22854...,"{'Total tokens': '8814', 'Input tokens': '2606...",2025-07-15 16:03:20.551411,<imageRequest>\n <metadata>\n <title>Formu...,motoring,approved,NaN
2,15046755,"Supercar Driving Experience – 3, 6 or 9 Laps |...",Activities,LOCAL,Motoring,161.19,3,608001,https://static.wowcher.co.uk/images/deal/15046...,jpg,success,https://static.wowcher.co.uk/images/deal/15046...,"{'Total tokens': '8846', 'Input tokens': '2638...",2025-07-15 16:03:20.736840,<imageRequest>\n <metadata>\n <title>Super...,motoring,approved,NaN
3,40376408,Ford Rally Car Driving Experience – 3 or 6 Mil...,Activities,LOCAL,Motoring,111.13,4,1608013,https://static.wowcher.co.uk/images/deal/40376...,jpg,success,https://static.wowcher.co.uk/images/deal/40376...,"{'Total tokens': '7418', 'Input tokens': '1210...",2025-07-15 16:03:20.931039,<imageRequest>\n <metadata>\n <title>Ford ...,motoring,approved,NaN
4,39868631,Nationwide Quad Biking Adventure for Two – Up ...,Activities,LOCAL,Motoring,105.83,5,1585635,https://static.wowcher.co.uk/images/deal/39868...,jpg,success,https://static.wowcher.co.uk/images/deal/39868...,"{'Total tokens': '7752', 'Input tokens': '1544...",2025-07-15 16:03:21.079676,<imageRequest>\n <metadata>\n <title>Natio...,motoring,approved,NaN


## 1. Load and filter approved images

In [22]:
import pandas as pd
import json
import re

# Function to safely parse the token info column
def extract_cost(token_info):
    try:
        # Handle if it's already a dict
        if isinstance(token_info, dict):
            return float(token_info.get('Cost', '0').replace('$', ''))
        
        # Try to parse as JSON if it's a string
        if isinstance(token_info, str):
            # Clean up any potential invalid JSON formatting
            cleaned = token_info.replace("'", '"')
            data = json.loads(cleaned)
            return float(data.get('Cost', '0').replace('$', ''))
    except:
        # If JSON parsing fails, try regex
        try:
            cost_match = re.search(r"'Cost': '\$([\d.]+)'", str(token_info))
            if cost_match:
                return float(cost_match.group(1))
        except:
            pass
    return 0.0

# Extract cost from token info column
df['extracted_cost'] = df['token_info'].apply(extract_cost)

# Calculate total cost and count by review result
cost_summary = df.groupby('review_result').agg(
    total_cost=('extracted_cost', 'sum'),
    count=('extracted_cost', 'count')
).reset_index()

# Format the cost with $ sign
cost_summary['total_cost'] = cost_summary['total_cost'].apply(lambda x: f"${x:.6f}")

print("Cost summary by approval status:")
print(cost_summary)

# Calculate overall total
overall_total_cost = df['extracted_cost'].sum()
overall_count = len(df)

print(f"\nOverall total cost: ${overall_total_cost:.6f}")
print(f"Overall count: {overall_count}")

Cost summary by approval status:
  review_result  total_cost  count
0      approved  $21.365415     82
1      rejected   $4.732040     18

Overall total cost: $26.097455
Overall count: 100


## 2. Prepare data for Redshift

In [23]:
# Prepare the data for Redshift
approved_df = df[(df['review_result'] == 'approved') & (df['image_id_pos_0'].notna()) & (df['image_id_pos_0'].apply(lambda x: isinstance(x, (int, float))))]
approved_df['image_id_pos_0'] = approved_df['image_id_pos_0'].apply(lambda x: int(x) if isinstance(x, float) else x)
redshift_df = prepare_for_redshift(approved_df)
# Display the first few rows of the prepared data
redshift_df.head()

Preparing data structure for Redshift...
Prepared 82 rows for processing


/var/folders/dk/zpttw0wn0192zryb4bdz19lr0000gn/T/ipykernel_55164/1705187957.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  approved_df['image_id_pos_0'] = approved_df['image_id_pos_0'].apply(lambda x: int(x) if isinstance(x, float) else x)


,deal_voucher_id,claid_prompt,status,original_image_id,variant_image_id,batch_name,enter_test_ts,exit_test_ts,list_name,s3_url,original_url,processed_status,final_s3_url,new_oracle_id
1,22854381,,1,852549,None,OPEN AI Images,2025-07-18 10:22:17,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/22854...,https://static.wowcher.co.uk/images/deal/22854...,False,None,None
2,15046755,,1,608001,None,OPEN AI Images,2025-07-18 10:22:17,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/15046...,https://static.wowcher.co.uk/images/deal/15046...,False,None,None
3,40376408,,1,1608013,None,OPEN AI Images,2025-07-18 10:22:17,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/40376...,https://static.wowcher.co.uk/images/deal/40376...,False,None,None
4,39868631,,1,1585635,None,OPEN AI Images,2025-07-18 10:22:17,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/39868...,https://static.wowcher.co.uk/images/deal/39868...,False,None,None
6,30129340,,1,1230732,None,OPEN AI Images,2025-07-18 10:22:17,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/30129...,https://static.wowcher.co.uk/images/deal/30129...,False,None,None


In [24]:
# Connect to Oracle and execute the SQL query
import oracledb

import os
from dotenv import load_dotenv

load_dotenv()

def get_oracle_connection():
    connection = oracledb.connect(
        user=os.getenv("ORACLE_USER"), 
        password=os.getenv("ORACLE_PASSWORD"), 
        dsn=os.getenv("ORACLE_DSN")
    )
    return connection

# Fetch data from Oracle
def fetch_data_from_oracle(deal_voucher_ids):
    query = """
    SELECT deal_voucher_id, id AS original_image_id
    FROM deal_voucher_image
    WHERE deal_voucher_id IN ({})
    AND position = 0
    """.format(','.join(map(str, deal_voucher_ids)))

    connection = get_oracle_connection()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    cursor.close()
    connection.close()
    return result

# Update redshift_df with data from Oracle
deal_voucher_ids = redshift_df['deal_voucher_id'].to_list()
oracle_data = fetch_data_from_oracle(deal_voucher_ids)

# Convert the fetched data to a DataFrame
oracle_df = pd.DataFrame(oracle_data, columns=['deal_voucher_id', 'original_image_id'])

# Merge the Oracle data with redshift_df
redshift_df = redshift_df.merge(oracle_df, on='deal_voucher_id', how='left', suffixes=('', '_oracle'))

# Update the original_image_id in redshift_df with the one from Oracle if available
redshift_df['original_image_id'] = redshift_df['original_image_id_oracle'].combine_first(redshift_df['original_image_id'])

# Drop the temporary column
redshift_df.drop(columns=['original_image_id_oracle'], inplace=True)


In [25]:
len(redshift_df)

82

## 3. Copy variants to S3 with new variant IDs

In [26]:
display(redshift_df)
processed_df = process_approved_variants_with_oracle(redshift_df)
# Display a summary of the processed data
success_count = processed_df['processed_status'].sum()
print(f"Successfully processed {success_count} of {len(processed_df)} variants")
redshift_df.to_csv('redshift_df.csv', index=False)

,deal_voucher_id,claid_prompt,status,original_image_id,variant_image_id,batch_name,enter_test_ts,exit_test_ts,list_name,s3_url,original_url,processed_status,final_s3_url,new_oracle_id
0,22854381,,1,852549,None,OPEN AI Images,2025-07-18 10:22:17,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/22854...,https://static.wowcher.co.uk/images/deal/22854...,False,None,None
1,15046755,,1,608001,None,OPEN AI Images,2025-07-18 10:22:17,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/15046...,https://static.wowcher.co.uk/images/deal/15046...,False,None,None
2,40376408,,1,1608013,None,OPEN AI Images,2025-07-18 10:22:17,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/40376...,https://static.wowcher.co.uk/images/deal/40376...,False,None,None
3,39868631,,1,1585635,None,OPEN AI Images,2025-07-18 10:22:17,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/39868...,https://static.wowcher.co.uk/images/deal/39868...,False,None,None
4,30129340,,1,1230732,None,OPEN AI Images,2025-07-18 10:22:17,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/30129...,https://static.wowcher.co.uk/images/deal/30129...,False,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,18275239,,1,721401,None,OPEN AI Images,2025-07-18 10:22:17,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/18275...,https://static.wowcher.co.uk/images/deal/18275...,False,None,None
78,32582607,,1,1345510,None,OPEN AI Images,2025-07-18 10:22:17,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/32582...,https://static.wowcher.co.uk/images/deal/32582...,False,None,None
79,40425836,,1,1610392,None,OPEN AI Images,2025-07-18 10:22:17,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/40425...,https://static.wowcher.co.uk/images/deal/40425...,False,None,None
80,40823928,,1,1656742,None,OPEN AI Images,2025-07-18 10:22:17,None,imgv_list_wow_uk,https://static.wowcher.co.uk/images/deal/40823...,https://static.wowcher.co.uk/images/deal/40823...,False,None,None


Processing approved variants with Oracle...
Processing approved variant for deal 22854381...
Processing approved variant for deal 15046755...
Processing approved variant for deal 40376408...
Processing approved variant for deal 39868631...
Processing approved variant for deal 30129340...
Processing approved variant for deal 40798451...
Processing approved variant for deal 18323724...
Processing approved variant for deal 38032495...
Processing approved variant for deal 36585301...
Processing approved variant for deal 25649163...
Processing approved variant for deal 40425842...
Processing approved variant for deal 39262480...
Processing approved variant for deal 22772210...
Processing approved variant for deal 20840452...
Processing approved variant for deal 39696346...
Processing approved variant for deal 18275242...
Processing approved variant for deal 40881741...
Processing approved variant for deal 28578269...
Processing approved variant for deal 28607280...
Processing approved varia

Processing Approved Variants with Oracle:   0%|          | 0/82 [00:00<?, ?it/s]

Got new Oracle image ID: 1706471
Got new Oracle image ID: 1706472Got new Oracle image ID: 1706474

Got new Oracle image ID: 1706475
Got new Oracle image ID: 1706473
Got new Oracle image ID: 1706479
Got new Oracle image ID: 1706478
Got new Oracle image ID: 1706476
Got new Oracle image ID: 1706477
Got new Oracle image ID: 1706483
Got new Oracle image ID: 1706480
Got new Oracle image ID: 1706481
Got new Oracle image ID: 1706482
Got new Oracle image ID: 1706484
Got new Oracle image ID: 1706488
Got new Oracle image ID: 1706485
Got new Oracle image ID: 1706486
Got new Oracle image ID: 1706489
Got new Oracle image ID: 1706487
Got new Oracle image ID: 1706490
Got new Oracle image ID: 1706491
Got new Oracle image ID: 1706494
Got new Oracle image ID: 1706492
Got new Oracle image ID: 1706493
Got new Oracle image ID: 1706495
Copied: images/deal/39868631/1585635-bonus.jpg -> images/deal/39868631/1706471-bonus.jpg
Copied: images/deal/39868631/1585635-bonus2.jpg -> images/deal/39868631/1706471-bonus2

Processing Approved Variants with Oracle:   1%|          | 1/82 [00:04<05:29,  4.07s/it]

...

Copied: images/deal/40823928/1656742-iphone-promo.jpg -> images/deal/40823928/1706552-iphone-promo.jpg
Copied: images/deal/18275239/721401-promo.jpg -> images/deal/18275239/1706548-promo.jpg
Copied: images/deal/40973727/1670326-iphone-medium.jpg -> images/deal/40973727/1706551-iphone-medium.jpg
Copied: images/deal/15852296/635838-user.jpg -> images/deal/15852296/1706547-user.jpg
Copied: images/deal/18275239/721401-thumb.jpg -> images/deal/18275239/1706548-thumb.jpg
Copied: images/deal/32582607/1345510-iphone-thumb.jpg -> images/deal/32582607/1706549-iphone-thumb.jpg
Copied: images/deal/40823928/1656742-iphone-small.jpg -> images/deal/40823928/1706552-iphone-small.jpg
✅ Successfully copied variant to S3 for deal 28362609: https://static.wowcher.co.uk/images/deal/28362609/170654500000.jpg
   (Variant ID 170654500000 exists only in S3, not in Oracle)
✅ Successfully copied variant to S3 for deal 37655990: https://static.wowcher.co.uk/images/deal/37655990/170654600000.jpg
   (Variant ID 170

Processing Approved Variants with Oracle:  95%|█████████▌| 78/82 [00:10<00:00,  9.28it/s]

Copied: images/deal/40425836/1610392-user.jpg -> images/deal/40425836/1706550-user.jpg
✅ Successfully copied variant to S3 for deal 15852296: https://static.wowcher.co.uk/images/deal/15852296/170654700000.jpg
   (Variant ID 170654700000 exists only in S3, not in Oracle)
Copied: images/deal/40973727/1670326.jpg -> images/deal/40973727/1706551.jpg
Skipping _variant file: images/deal/40973727/1670326_variant.jpg
Copied 14 existing files
✅ Successfully copied variant to S3 for deal 18275239: https://static.wowcher.co.uk/images/deal/18275239/170654800000.jpg
   (Variant ID 170654800000 exists only in S3, not in Oracle)
Copied: images/deal/40425836/1610392.jpg -> images/deal/40425836/1706550.jpg
Skipping _variant file: images/deal/40425836/1610392_variant.jpg
Copied 14 existing files
✅ Replaced 1 Oracle records with new ID 1706552, keeping original positions
✅ Replaced 1 Oracle records with new ID 1706549, keeping original positions
✅ Replaced 1 Oracle records with new ID 1706551, keeping or

Processing Approved Variants with Oracle: 100%|██████████| 82/82 [00:11<00:00,  7.17it/s]

✅ Successfully copied variant to S3 for deal 32582607: https://static.wowcher.co.uk/images/deal/32582607/170654900000.jpg
   (Variant ID 170654900000 exists only in S3, not in Oracle)
✅ Successfully copied variant to S3 for deal 40973727: https://static.wowcher.co.uk/images/deal/40973727/170655100000.jpg
   (Variant ID 170655100000 exists only in S3, not in Oracle)
✅ Successfully copied variant to S3 for deal 40425836: https://static.wowcher.co.uk/images/deal/40425836/170655000000.jpg
   (Variant ID 170655000000 exists only in S3, not in Oracle)
Successfully processed 82 of 82 variants
Saved processed data to processed_approved_variants.csv
Successfully processed 82 of 82 variants


## 4. Upload prepared data to S3 for Redshift

In [27]:
# Upload the prepared data to S3 for Redshift
s3_url = upload_to_s3_for_redshift(processed_df)

if s3_url:
    print(f"Successfully uploaded data to S3: {s3_url}")
else:
    print("Failed to upload data to S3")

Preparing data for Redshift...
Found 82 successfully processed variants for Redshift
Saved Redshift data to redshift_upload_data.csv
Uploaded data to S3: https://static.wowcher.co.uk/temp/ai_image_variants_20250718102252.csv
Successfully uploaded data to S3: https://static.wowcher.co.uk/temp/ai_image_variants_20250718102252.csv


## 5. Copy data from S3 to Redshift

In [28]:
# Only proceed if we have a valid S3 URL
if s3_url:
    # Copy the data from S3 to Redshift
    success = copy_s3_to_redshift(s3_url)
    if success:
        print("Successfully copied data to Redshift")
    else:
        print("Failed to copy data to Redshift")
else:
    print("Skipping Redshift update due to missing S3 URL")

Copying data to Redshift...
Successfully copied 6467 rows to Redshift table
Successfully copied data to Redshift


## 6. Update the image list in the API

In [29]:
# Only proceed if we successfully copied to Redshift
if 'success' in locals() and success:
    # Update the image list in the API
    result = update_image_list()
    if result:
        print("Successfully updated the image list in the API")
    else:
        print("Failed to update the image list in the API")
else:
    print("Skipping API update due to failed Redshift update")

Updating image list in API...
Found 5060 unique NEW variant images to add to list
Successfully updated test list with 5060 NEW variant images
Successfully updated the image list in the API


# Purge Cache on image urls. Usefull if making images where a variant already existed and now need it to show the new variant


In [30]:
ids_str = df['id'].apply(lambda x: str(int(x))).to_list()
print('re-index in chunks of 50 at this url: https://search-ingest-gr.wowcher.co.uk/')
# Iterate over the list in chunks of 50
for i in range(0, len(ids_str), 30):
    # Join the chunk into a comma-separated string and print
    print(','.join(ids_str[i:i+30]))


re-index in chunks of 50 at this url: https://search-ingest-gr.wowcher.co.uk/
16268711,22854381,15046755,40376408,39868631,40435125,30129340,40798451,18323724,23449742,38032495,36585301,22772241,20861597,28813228,36733612,25649163,28507086,40425842,15852251,31524822,39262480,22772210,20840452,39696346,18275242,40881741,28578269,28607280,28619855
40488761,28363060,38032117,38032069,20012983,20205662,15038111,16165901,16254629,16261285,27636368,30348624,32910355,32910944,22825312,24085229,36566050,39250834,26053323,39696402,28620033,29754700,30382838,39770162,31461035,16268616,18083904,26423830,38117224,39705204
15940162,27604653,22772128,31461193,40065180,26925407,37181969,16576247,20952110,25677233,30624403,32615119,25415406,27604710,28520307,36622907,40435176,37670759,16261669,20028154,20820344,27791950,28578977,29797372,30303592,32614825,33794787,39759396,18275160,21849899
24085171,26053380,28362609,37655990,15852296,18275239,32582607,40425836,40823928,40973727


In [15]:
import psycopg2

conn_params = {
    "host": "bi-redshift.intwowcher.co.uk",
    "port": 5439,
    "dbname": "wowdwhprod",
    "user": "jenkins",
    "password": "9SDy1ffdfTV7"
}

def cleanup_images():
    sequence_query = """
    UPDATE temp.opt_image_variants t1
    SET 
        status = 5,
        exit_test_ts = t2.enter_test_ts
    FROM temp.opt_image_variants t2
    WHERE (
        (t1.original_image_id = t2.original_image_id AND t1.id < t2.id)
        OR
        (t1.deal_voucher_id = t2.deal_voucher_id AND t1.original_image_id != t2.original_image_id AND t1.batch_name= 'OPEN AI Images' AND t2.batch_name= 'OPEN AI Images')
    )
    AND t1.status = 1 
    AND t2.status = 1;
    """
    with psycopg2.connect(**conn_params) as conn:
        with conn.cursor() as cur:
            cur.execute(sequence_query)
            row_count = cur.rowcount
            conn.commit()
            print(f"Successfully updated {row_count} earlier variants with exit_test_ts and status=5")
            print("Latest variants for each deal remain unchanged with status=1")

# Execute the bulk update
sequence_results = cleanup_images()

Successfully updated 0 earlier variants with exit_test_ts and status=5
Latest variants for each deal remain unchanged with status=1


In [ ]:
import psycopg2
import pandas as pd

sql = """

select 
'https://static.wowcher.co.uk/images/deal/'||oiv.deal_voucher_id||'/'||oiv.original_image_id||'00000.'||dvi.extension as url
from temp.opt_image_variants oiv
join real.deal_voucher_image dvi on dvi.id = oiv.original_image_id
where oiv.status = 1
and trunc(enter_test_ts) = trunc(sysdate)
"""

REDSHIFT_CONFIG = {
    'host': 'bi-redshift.intwowcher.co.uk',
    'port': 5439,
    'dbname': 'wowdwhprod',
    'user': 'jenkins',
    'password': '9SDy1ffdfTV7'
}


conn = psycopg2.connect(**REDSHIFT_CONFIG)

# Use pandas to read SQL query
urls =  pd.read_sql_query(sql, conn)['url'].to_list()

# Close connection
conn.close()

In [ ]:
import requests 
import json

urls = ['https://www.wowcher.co.uk/deals/travel/european-city-breaks']
def purge_cache(image_urls):
    chunk_size = 30 # cloudflare purge api takes max 30 urls per request
    chunked_list = [image_urls[i:i+chunk_size] for i in range(0, len(image_urls), chunk_size)]
    api_url = "https://api.cloudflare.com/client/v4/zones/4fec7e02d5c45deb9f67452873708896/purge_cache"
    api_key = "Bearer IT-Lr8A8LOipKdcGKkJU8Q5ciM8jfy3KbtfWkDnK"
    headers = {"Authorization": api_key,
               "Content-Type":"application/json"
               }
    for chunk in chunked_list:
        payload = {"files": []}
        #print("Clearing cache of: ", chunk)
        payload['files'] = chunk
        response = requests.post(url=api_url, data=json.dumps(payload), headers=headers)
        print(response.json())
purge_cache(urls)